In [ ]:
pip install geojson-validator fiona pyproj

## Imports

In [ ]:
import os
import shutil
from osgeo import ogr, osr

## to_geojson()

This function converts the MRM output JSON records to valid geojson structure, in WGS84 geographic coordinates.

# Mostly not working properly

In [ ]:
import os
from osgeo import ogr, osr

def convert_to_wgs84(directory_path, output_path):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".geojson"):
            filepath = os.path.join(directory_path, filename)
            
            # Open the input GeoJSON file
            driver = ogr.GetDriverByName('GeoJSON')
            datasource = driver.Open(filepath, 0)  # 0 means read-only
            
            if datasource is None:
                print(f"Could not open {filename}")
                continue
            
            layer = datasource.GetLayer()
            
            # Define the target spatial reference (WGS84)
            target_srs = osr.SpatialReference()
            target_srs.ImportFromEPSG(4326)
            
            # Define the source spatial reference (WGS84)
            source_srs = osr.SpatialReference()
            source_srs.ImportFromEPSG(3857)
            transform = osr.CoordinateTransformation(source_srs, target_srs)
            
            # Create the output datasource
            output_filepath = os.path.join(output_path, filename)
            out_datasource = driver.CreateDataSource(output_filepath)
            out_layer = out_datasource.CreateLayer(layer.GetName(), target_srs, ogr.wkbUnknown)
            
            # Add fields from the input layer to the output layer
            layer_defn = layer.GetLayerDefn()
            for i in range(layer_defn.GetFieldCount()):
                field_defn = layer_defn.GetFieldDefn(i)
                out_layer.CreateField(field_defn)
            
            # Add a new field for the filename
            new_field = ogr.FieldDefn("filename", ogr.OFTString)
            out_layer.CreateField(new_field)
            
            # Get the output layer definition
            out_layer_defn = out_layer.GetLayerDefn()
            
            # Loop through the input features
            for feature in layer:
                geom = feature.GetGeometryRef()
                
                if geom is not None:
                    # Transform the geometry to WGS84
                    geom.Transform(transform)
                
                    # Create a new feature for the output layer
                    out_feature = ogr.Feature(out_layer_defn)
                    out_feature.SetGeometry(geom)
                    
                    # Copy attributes from the input feature to the output feature
                    for i in range(out_layer_defn.GetFieldCount() - 1):  # Exclude the last field (filename)
                        out_feature.SetField(out_layer_defn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))
                    
                    # Set the filename attribute
                    out_feature.SetField("filename", filename)
                    
                    # Add the feature to the output layer
                    out_layer.CreateFeature(out_feature)
                    
                    # Destroy the feature to free resources
                    out_feature.Destroy()
            
            # Close the datasources
            datasource.Destroy()
            out_datasource.Destroy()

            print(f"Processed {filename} and saved to {output_filepath}")


# Example usage
convert_to_wgs84('./data/testing_subset/originals_testing/', './data/testing_subset/wgs84_geojson/')

## separate_geometries()

In [ ]:


def is_valid_geometry(geometry):
    # Check if the geometry is within valid lat/long bounds
    if geometry is None:
        return False

    for ring in range(geometry.GetGeometryCount()):
        sub_geom = geometry.GetGeometryRef(ring)
        for point in range(sub_geom.GetPointCount()):
            lon, lat, _ = sub_geom.GetPoint(point)
            if not (-180 <= lon <= 180 and -90 <= lat <= 90):
                return False
    return True

def separate_geometries(directory_path, valid_output_path, invalid_output_path):
    # Ensure output directories exist
    if not os.path.exists(valid_output_path):
        os.makedirs(valid_output_path)
    if not os.path.exists(invalid_output_path):
        os.makedirs(invalid_output_path)
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".geojson"):
            filepath = os.path.join(directory_path, filename)
            
            # Open the input GeoJSON file
            driver = ogr.GetDriverByName('GeoJSON')
            datasource = driver.Open(filepath, 0)  # 0 means read-only
            
            if datasource is None:
                print(f"Could not open {filename}")
                continue
            
            layer = datasource.GetLayer()
            
            # Create output datasources for valid and invalid geometries
            valid_out_filepath = os.path.join(valid_output_path, filename)
            invalid_out_filename = filename.replace(".geojson", "_invalid_geometries.geojson")
            invalid_out_filepath = os.path.join(invalid_output_path, invalid_out_filename)
            
            valid_out_datasource = driver.CreateDataSource(valid_out_filepath)
            invalid_out_datasource = driver.CreateDataSource(invalid_out_filepath)
            
            # Set the same layer definition for valid and invalid output layers
            valid_out_layer = valid_out_datasource.CreateLayer(layer.GetName(), layer.GetSpatialRef(), ogr.wkbUnknown)
            invalid_out_layer = invalid_out_datasource.CreateLayer(layer.GetName(), layer.GetSpatialRef(), ogr.wkbUnknown)
            
            # Add fields from the input layer to both output layers
            layer_defn = layer.GetLayerDefn()
            for i in range(layer_defn.GetFieldCount()):
                field_defn = layer_defn.GetFieldDefn(i)
                valid_out_layer.CreateField(field_defn)
                invalid_out_layer.CreateField(field_defn)
            
            # Add a new field for the filename
            new_field = ogr.FieldDefn("filename", ogr.OFTString)
            valid_out_layer.CreateField(new_field)
            invalid_out_layer.CreateField(new_field)
            
            # Get the output layer definitions
            valid_out_layer_defn = valid_out_layer.GetLayerDefn()
            invalid_out_layer_defn = invalid_out_layer.GetLayerDefn()
            
            # Loop through the input features
            for feature in layer:
                geom = feature.GetGeometryRef()
                
                if is_valid_geometry(geom):
                    out_layer = valid_out_layer
                    out_layer_defn = valid_out_layer_defn
                else:
                    out_layer = invalid_out_layer
                    out_layer_defn = invalid_out_layer_defn
                
                # Create a new feature for the output layer
                out_feature = ogr.Feature(out_layer_defn)
                out_feature.SetGeometry(geom)
                
                # Copy attributes from the input feature to the output feature
                for i in range(out_layer_defn.GetFieldCount() - 1):  # Exclude the last field (filename)
                    out_feature.SetField(out_layer_defn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))
                
                # Set the filename attribute
                out_feature.SetField("filename", filename)
                
                # Add the feature to the output layer
                out_layer.CreateFeature(out_feature)
                
                # Destroy the feature to free resources
                out_feature.Destroy()
            
            # Close the datasources
            datasource.Destroy()
            valid_out_datasource.Destroy()
            invalid_out_datasource.Destroy()

            print(f"Processed {filename}, valid geometries saved to {valid_out_filepath}, invalid geometries saved to {invalid_out_filepath}")

# Example usage
separate_geometries('./data/testing_subset/wgs84_geojson/', './data/testing_subset/valid_wgs84_geometries', './data/testing_subset/invalid_wgs84_geometries')

## 